In [1]:
import json

In [2]:
with open('../Data/Babelio/item.json') as json_data:
    babelioData = json.load(json_data)

In [3]:
print(babelioData[2])

{'url': 'https://www.babelio.com/livres/Rivard-La-Complainte-du-phoque-en-Alaska/461730', 'title': ' La Complainte du phoque en Alaska ', 'author': ['Michel ', 'Isabelle ', 'Rivard', 'Chatellard'], 'author_id': '/auteur/Michel-Rivard/258147', 'infos': ' EAN : 9782278053346   24 pages            Éditeur : Didier Jeunesse  (23/04/2003)   ', 'editor': 'Didier Jeunesse', 'rating': None, 'nb_rating': None, 'resume': [' La complainte du phoque en Alaska est une chanson qui fut un succès pour le groupe québécois "Beau Dommage" en 1974. Cette fable québécoise évoque un amour mis à mal par le rêve américain sous les traits d\'un phoque abandonné par sa compagne pour aller gagner sa vie dans un cirque aux Etats-Unis...  '], 'tags': [{'tag': ' fables  ', 'info': 'tag_t14 tc_0 tc_noreco00 tc_noaff00 tc_bold ', 'importance': '14'}, {'tag': ' littérature jeunesse  ', 'info': 'tag_t14 tc_0 tc_noreco00 tc_noaff00 tc_bold ', 'importance': '14'}, {'tag': ' jeunesse  ', 'info': 'tag_t14 tc_0 tc_noreco00 

In [4]:
class Livre:
     def __init__(self, titre = "***NO TITLE***"):
        self.titre = titre
        self.rating = None
        self.annee = ""
        self.auteur = ""
        self.isbn = []
        self.tags = []
        self.editeur = None
        self.url = ""
        
class Tag:
    def __init__(self, name, info, importance):
        self.name = name
        self.info = info
        self.importance = importance

In [5]:
def extraireTitre(string):
    return  string.split(' / ')[0].split(' : ')[0]

def extraireAnnee(string):
    return  string.split(' / ')[-1]

In [13]:
livres = []
error = []

for data in babelioData:
    try :
        livre = Livre(extraireTitre(data["title"]))
        livre.id = babelioData.index(data)
        livre.annee = data['edition_date']
        livre.isbn = data['EAN']
        livre.auteur = data['author']
        livre.editeur = data['editor']
        livre.url = data['url']
        for t in data['tags']:
            livre.tags.append(Tag(t['tag'], t['info'], t['importance']))

        livres.append(livre)
    except:
        error.append('')
print(len(livres))
print(len(error))

1793
1568


# Création du graphe RDF

In [7]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD

SCHEMA = Namespace("https://schema.org/")
DBO = Namespace("http://dbpedia.org/ontology/")


In [8]:
grapheBabelio = Graph()

def creerUriAuteur(auteur):
    return SCHEMA['auteur/' + str(auteur.id)]

# for a in auteurs:
    

for l in livres:
    #TODO check avec Gregoire si ID existe sur Babelio, sinon creer un
    livre_uri = SCHEMA['livre/' + str(l.id)]
    grapheBabelio.add((livre_uri, SCHEMA.name, Literal(l.titre)))
    grapheBabelio.add((livre_uri, RDF.type, SCHEMA['Book']))
    grapheBabelio.add((livre_uri, SCHEMA.isbn, Literal(l.isbn)))

    grapheBabelio.add((livre_uri, SCHEMA.author, Literal(l.auteur)))

    grapheBabelio.add((livre_uri, SCHEMA.datePublished, Literal(l.annee)))

#     grapheBabelio.add((livre_uri, SCHEMA.publisher, creerUriEditeur(l.editeur)))
    grapheBabelio.add((livre_uri, SCHEMA.publisher, Literal(l.editeur)))



# Sauvegarde du graphe

In [9]:
fichier = open("../Graphes/grapheBabelio.rdf","wb")
grapheBabelio.serialize(fichier) 
fichier.close()